evaluation low

In [29]:
import torch
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from data_utils import mnist_preparation, add_salt_and_pepper_noise 
from evaluationUtils import calculate_mcm_accuracy
from tqdm import tqdm
import matplotlib as plt
from mcmadaptablemodel import MCMQuantumModel, MCMCircuit
from pennylane import Device
from pennylane.measurements import StateMP
from torch.nn import Module, ParameterDict
import matplotlib.pyplot as plt
from OriginalModel import FullQuantumModel, QuantumCircuit
import warnings
from typing import Optional, Dict, List, Any
from torch.utils.data import DataLoader, dataloader
from time import time
import math
from pennylane.measurements import MidMeasureMP
torch.manual_seed(1234)

In [30]:
def early_evaluation_utils(params: Dict, state: torch.Tensor = None): 
    first_pair = [0,1]
    measurements = []
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(params[f'layer_{i}'][j, 0], wires=j)
            qml.RY(params[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(params[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
    
    for w in first_pair: 
        measurements.append(qml.measure(wires=w)) #measure first pair of qubits
    return measurements

def fully_evaluation_utils(params: Dict, state: torch.Tensor = None):
    first_pair = [0,1]
    second_pair = [2,3]
    mcasurements = []
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(params[f'layer_{i}'][j, 0], wires=j)
            qml.RY(params[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(params[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            
    for w in first_pair: 
        mcasurements.append(qml.measure(wires=w)) #measure first pair of qubits

    for i in range(4, 8):
        for j in range(8):
            qml.RX(params[f'layer_{i}'][j, 0], wires=j)
            qml.RY(params[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(params[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])

    for w in second_pair:
        mcasurements.append(qml.measure(wires=w))

    return mcasurements

dev = qml.device("default.qubit", wires=8)
@qml.qnode(dev)  
def early_evaluation_ansatz(params: Dict, state: torch.Tensor = None):
    early_measurement = early_evaluation_utils(params=params, state=state)
    return qml.probs(op=early_measurement)

@qml.qnode(dev)
def fully_evaluation_ansatz(params: Dict, state: torch.Tensor = None):
    measurements = fully_evaluation_utils(params=params, state=state)
    mid_measurement = measurements[:2]
    final_measurement = measurements[2:]
    return qml.probs(op=mid_measurement), qml.probs(op=final_measurement)

In [31]:
def evaluation_routine(dataloader: DataLoader, parameters: Dict, threshold: float):
    
    evaluation_results = []
    early_results = []
    count_1 = 0 #counter for early classified images
    count_2 = 0 #counter for final classified images
    early_correct = 0 #counter for correctly early classified images 
    final_correct = 0 #counter for correctly final classified images
    executed_layers = 0
    for img, target in dataloader.dataset:
        #img normalization
        img = img / torch.linalg.norm(img).view(-1, 1)
        
        #mid circuit evaluation
        early_probs = early_evaluation_ansatz(params=parameters, state=img)
        early_prediction = torch.argmax(early_probs, dim=1)
        confidence = early_probs[0, early_prediction].item()
        early_guess = early_prediction == target
        early_results.append(early_guess.item())
        
        if confidence >= threshold:
            evaluation_results.append(early_guess.item())
            count_1 += 1
            executed_layers += 4
            if early_guess: 
                early_correct += 1
            
        else: 
            final_probs = fully_evaluation_ansatz(params=parameters, state=img)
            early_full, final_full = final_probs
            final_predictions = torch.argmax(final_full, dim=1)
            final_guess = final_predictions == target
            evaluation_results.append(final_guess.item())
            count_2 += 1
            executed_layers += 12
            
            if final_guess: 
                final_correct += 1
    total_accuracy = sum([1 for i in evaluation_results if i == True])/len(evaluation_results)
    early_total_accuracy = sum([1 for i in early_results if i == True])/len(early_results)
    early_exited_accuracy = early_correct/count_1 if count_1 > 0 else 0
    final_exited_accuracy = final_correct/count_2 if count_2 > 0 else 0
    
    return total_accuracy, early_total_accuracy, early_exited_accuracy, count_1, final_exited_accuracy, count_2, executed_layers

In [32]:
def explain_evaluation(dataloader: DataLoader, parameters: Dict, threshold: List[float]):
    summary_data = {
    'Threshold': [],
    'Total Accuracy': [],
    '# early exited images': [],
    'Early exited Accuracy': [],
    'Early total accuracy': [],
    '# final classified images': [],
    'Final classified Accuracy': [],
    "Executed layers": []}
    
    for t in tqdm(threshold):
        total_accuracy, early_total_accuracy, early_exited_accuracy, count_1, final_exited_accuracy, count_2, executed_layers = evaluation_routine(dataloader, parameters, t)
        summary_data['Threshold'].append(t)
        summary_data['Total Accuracy'].append(total_accuracy)
        summary_data['# early exited images'].append(count_1)
        summary_data['Early exited Accuracy'].append(early_exited_accuracy)
        summary_data['Early total accuracy'].append(early_total_accuracy)
        summary_data['# final classified images'].append(count_2)
        summary_data['Final classified Accuracy'].append(final_exited_accuracy)
        summary_data['Executed layers'].append(executed_layers)
        
    df = pd.DataFrame(summary_data)
    return summary_data, df

In [33]:
labels = [0,1,2,3]
# Download MNIST and prepare transforms
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                                transforms.Resize((16, 16)),  # Resize to 16x16
                                transforms.ToTensor(),
                                transforms.Lambda(lambda img: add_salt_and_pepper_noise(img, salt_prob=0.1, pepper_prob=0.1)),
                                transforms.Normalize((0.1307,), (0.3081,))  # Normalize
                             ]))
#train/vali/test 70/15/15 split, see data_utils.py for further details
train_dataloader, validation_dataloader, test_dataloader = mnist_preparation(dataset=mnist, labels = labels, train_test_ratio=0.7,batch_size=64, vali_test_ratio=0.5)

print("Images in the training set: ", len(train_dataloader.dataset), "\n Images in the validation set: ", len(validation_dataloader.dataset), "\n Images in the test set: ", len(test_dataloader.dataset))

In [34]:
import pickle
with open("/Users/jackvittori/Desktop/lownoise/traininghistory.pickle", "rb") as file: 
    training_history = pickle.load(file)
    
loss_history = training_history['loss_history']
mcm_accuracy = training_history['mcm_accuracy']
fm_accuracy = training_history['fm_accuracy']
weights = training_history['model_params']

In [35]:
plt.style.use('ggplot')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))

# Plotting the loss on the first subplot
ax1.plot(list(range(50)), loss_history, marker='.', linestyle='-', color='b',
         label='Loss per Epoch')
ax1.set_title('Training Loss Over Epochs', fontsize=16)
ax1.set_xlabel('Epochs', fontsize=14)
ax1.set_ylabel('Loss', fontsize=14)
ax1.set_xticks(list(range(0, 50, 2)))  # Mostra i tick ogni 5 epochs per ridurre la sovrapposizione
ax1.set_ylim(2.4, 2.8)  # Fissa la scala dell'asse y tra 2 e 3
ax1.legend()
ax1.grid(True)

# Plotting mcm_accuracy and fm_accuracy on the same plot (ax2)
ax2.plot(list(range(50)), mcm_accuracy, marker='.', linestyle='--', color='r',
         label='Mid circuit accuracy per epoch')
ax2.plot(list(range(50)), fm_accuracy, marker='.', linestyle='--', color='g',
         label='Final circuit accuracy per epoch')
ax2.set_title('Training Accuracy Over Epochs', fontsize=16)
ax2.set_xlabel('Epochs', fontsize=14)
ax2.set_ylabel('Accuracy', fontsize=14)
ax2.set_xticks(list(range(0, 50, 2)))  # Mostra i tick ogni 5 epochs per ridurre la sovrapposizione
ax2.set_ylim(0.20, 0.95)  # Fissa la scala dell'asse y tra 0.20 e 0.95
ax2.legend()
ax2.grid(True)


plt.tight_layout()
plt.savefig('/Users/jackvittori/Desktop/lownoise/training010.png', dpi=300)
plt.show()

In [36]:
early_probs_distribution = []
final_probs_distribution = []
for img, target in tqdm(validation_dataloader.dataset): 
    img = img / torch.linalg.norm(img).view(-1, 1)
    evaluation = fully_evaluation_ansatz(params=weights, state=img)
    early_full, final_full = evaluation
    
    early_prediction = torch.argmax(early_full, dim=1)
    early_probs = early_full[0, early_prediction].item()
    
    final_prediction = torch.argmax(final_full, dim=1)
    final_probs = final_full[0, final_prediction].item()
    
    early_probs_distribution.append(early_probs)
    final_probs_distribution.append(final_probs)

plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(10, 5))

# Creazione dell'istogramma per il primo array
ax.hist(early_probs_distribution, bins=60, density=False, alpha=0.5, color='g', edgecolor='black', label='Mid circuit probability distribution')

# Creazione dell'istogramma per il secondo array
ax.hist(final_probs_distribution, bins=60, density=False, alpha=0.5, color='r', edgecolor='black', label='final circuit probability distribution')

# Titolo e etichette degli assi
ax.set_title('Prediction confidence distribution', fontsize=16)
ax.set_xlabel('Prediction confidence', fontsize=14)
ax.set_ylabel('Occurrencies', fontsize=14)

# Aggiunta della legenda per distinguere le distribuzioni
ax.legend()

# Griglia e layout
ax.grid(True)
plt.tight_layout()

plt.savefig('/Users/jackvittori/Desktop/lownoise/probability_distribution01.png', dpi=300)
plt.show()

In [37]:
thresholds = [round(x * 0.01 + 0.26, 2) for x in range(26)]
summary, table = explain_evaluation(validation_dataloader, weights, thresholds)

In [38]:
ev_data = {
    'summary': summary,
    'table': table}

with open("/Users/jackvittori/Desktop/lownoise/evaluation01.pickle", "wb") as file:
    pickle.dump(ev_data, file)

# MID NOISE

In [56]:
import torch
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from data_utils import mnist_preparation, add_salt_and_pepper_noise 
from evaluationUtils import calculate_mcm_accuracy
from tqdm import tqdm
import matplotlib as plt
from mcmadaptablemodel import MCMQuantumModel, MCMCircuit
from pennylane import Device
from pennylane.measurements import StateMP
from torch.nn import Module, ParameterDict
import matplotlib.pyplot as plt
from OriginalModel import FullQuantumModel, QuantumCircuit
import warnings
from typing import Optional, Dict, List, Any
from torch.utils.data import DataLoader, dataloader
from time import time
import math
from pennylane.measurements import MidMeasureMP
torch.manual_seed(1234)

In [57]:
labels = [0,1,2,3]
# Download MNIST and prepare transforms
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                                transforms.Resize((16, 16)),  # Resize to 16x16
                                transforms.ToTensor(),
                                transforms.Lambda(lambda img: add_salt_and_pepper_noise(img, salt_prob=0.15, pepper_prob=0.15)),
                                transforms.Normalize((0.1307,), (0.3081,))  # Normalize
                             ]))
#train/vali/test 70/15/15 split, see data_utils.py for further details
train_dataloader, validation_dataloader, test_dataloader = mnist_preparation(dataset=mnist, labels = labels, train_test_ratio=0.7,batch_size=64, vali_test_ratio=0.5)

print("Images in the training set: ", len(train_dataloader.dataset), "\n Images in the validation set: ", len(validation_dataloader.dataset), "\n Images in the test set: ", len(test_dataloader.dataset))

In [59]:
import pickle
with open("/Users/jackvittori/Desktop/midnoise/file_con_pesi_su_cpu.pkl", "rb") as file:
    training_history = pickle.load(file)
    
loss_history = training_history['loss_history']
mcm_accuracy = training_history['mcm_accuracy']
fm_accuracy = training_history['fm_accuracy']
weights = training_history['model_params']

In [60]:
plt.style.use('ggplot')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))

# Plotting the loss on the first subplot
ax1.plot(list(range(50)), loss_history, marker='.', linestyle='-', color='b',
         label='Loss per Epoch')
ax1.set_title('Training Loss Over Epochs', fontsize=16)
ax1.set_xlabel('Epochs', fontsize=14)
ax1.set_ylabel('Loss', fontsize=14)
ax1.set_xticks(list(range(0, 50, 2)))  # Mostra i tick ogni 5 epochs per ridurre la sovrapposizione
ax1.set_ylim(2.4, 2.8)  # Fissa la scala dell'asse y tra 2 e 3
ax1.legend()
ax1.grid(True)

# Plotting mcm_accuracy and fm_accuracy on the same plot (ax2)
ax2.plot(list(range(50)), mcm_accuracy, marker='.', linestyle='--', color='r',
         label='Mid circuit accuracy per epoch')
ax2.plot(list(range(50)), fm_accuracy, marker='.', linestyle='--', color='g',
         label='Final circuit accuracy per epoch')
ax2.set_title('Training Accuracy Over Epochs', fontsize=16)
ax2.set_xlabel('Epochs', fontsize=14)
ax2.set_ylabel('Accuracy', fontsize=14)
ax2.set_xticks(list(range(0, 50, 2)))  # Mostra i tick ogni 5 epochs per ridurre la sovrapposizione
ax2.set_ylim(0.20, 0.95)  # Fissa la scala dell'asse y tra 0.20 e 0.95
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig('/Users/jackvittori/Desktop/midnoise/training015.png', dpi=300)
plt.show()

In [61]:
early_probs_distribution = []
final_probs_distribution = []
for img, target in tqdm(validation_dataloader.dataset): 
    img = img / torch.linalg.norm(img).view(-1, 1)
    evaluation = fully_evaluation_ansatz(params=weights, state=img)
    early_full, final_full = evaluation
    
    early_prediction = torch.argmax(early_full, dim=1)
    early_probs = early_full[0, early_prediction].item()
    
    final_prediction = torch.argmax(final_full, dim=1)
    final_probs = final_full[0, final_prediction].item()
    
    early_probs_distribution.append(early_probs)
    final_probs_distribution.append(final_probs)

plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(10, 5))

# Creazione dell'istogramma per il primo array
ax.hist(early_probs_distribution, bins=60, density=False, alpha=0.5, color='g', edgecolor='black', label='Mid circuit probability distribution')

# Creazione dell'istogramma per il secondo array
ax.hist(final_probs_distribution, bins=60, density=False, alpha=0.5, color='r', edgecolor='black', label='final circuit probability distribution')

# Titolo e etichette degli assi
ax.set_title('Prediction confidence distribution', fontsize=16)
ax.set_xlabel('Prediction confidence', fontsize=14)
ax.set_ylabel('Occurrencies', fontsize=14)

# Aggiunta della legenda per distinguere le distribuzioni
ax.legend()

# Griglia e layout
ax.grid(True)
plt.tight_layout()

plt.savefig('/Users/jackvittori/Desktop/midnoise/probability_distribution015.png', dpi=300)
plt.show()

In [62]:
thresholds = [round(x * 0.01 + 0.26, 2) for x in range(26)]
summary, table = explain_evaluation(validation_dataloader, weights, thresholds)

In [63]:
ev_data = {
    'summary': summary,
    'table': table}

with open("/Users/jackvittori/Desktop/midnoise/evaluation015.pickle", "wb") as file:
    pickle.dump(ev_data, file)

In [65]:
table

# HIGH NOISE

In [42]:
import torch
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from data_utils import mnist_preparation, add_salt_and_pepper_noise 
from evaluationUtils import calculate_mcm_accuracy
from tqdm import tqdm
import matplotlib as plt
from mcmadaptablemodel import MCMQuantumModel, MCMCircuit
from pennylane import Device
from pennylane.measurements import StateMP
from torch.nn import Module, ParameterDict
import matplotlib.pyplot as plt
from OriginalModel import FullQuantumModel, QuantumCircuit
import warnings
from typing import Optional, Dict, List, Any
from torch.utils.data import DataLoader, dataloader
from time import time
import math
from pennylane.measurements import MidMeasureMP
torch.manual_seed(1234)

In [43]:
labels = [0,1,2,3]
# Download MNIST and prepare transforms
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                                transforms.Resize((16, 16)),  # Resize to 16x16
                                transforms.ToTensor(),
                                transforms.Lambda(lambda img: add_salt_and_pepper_noise(img, salt_prob=0.2, pepper_prob=0.2)),
                                transforms.Normalize((0.1307,), (0.3081,))  # Normalize
                             ]))
#train/vali/test 70/15/15 split, see data_utils.py for further details
train_dataloader, validation_dataloader, test_dataloader = mnist_preparation(dataset=mnist, labels = labels, train_test_ratio=0.7,batch_size=64, vali_test_ratio=0.5)

print("Images in the training set: ", len(train_dataloader.dataset), "\n Images in the validation set: ", len(validation_dataloader.dataset), "\n Images in the test set: ", len(test_dataloader.dataset))

In [44]:
import pickle
with open("/Users/jackvittori/Desktop/highnoise/traininghistory.pickle", "rb") as file: 
    training_history = pickle.load(file)
    
loss_history = training_history['loss_history']
mcm_accuracy = training_history['mcm_accuracy']
fm_accuracy = training_history['fm_accuracy']
weights = training_history['model_params']

In [45]:
plt.style.use('ggplot')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))

# Plotting the loss on the first subplot
ax1.plot(list(range(50)), loss_history, marker='.', linestyle='-', color='b',
         label='Loss per Epoch')
ax1.set_title('Training Loss Over Epochs', fontsize=16)
ax1.set_xlabel('Epochs', fontsize=14)
ax1.set_ylabel('Loss', fontsize=14)
ax1.set_xticks(list(range(0, 50, 2)))  # Mostra i tick ogni 5 epochs per ridurre la sovrapposizione
ax1.set_ylim(2.4, 2.8)  # Fissa la scala dell'asse y tra 2 e 3
ax1.legend()
ax1.grid(True)

# Plotting mcm_accuracy and fm_accuracy on the same plot (ax2)
ax2.plot(list(range(50)), mcm_accuracy, marker='.', linestyle='--', color='r',
         label='Mid circuit accuracy per epoch')
ax2.plot(list(range(50)), fm_accuracy, marker='.', linestyle='--', color='g',
         label='Final circuit accuracy per epoch')
ax2.set_title('Training Accuracy Over Epochs', fontsize=16)
ax2.set_xlabel('Epochs', fontsize=14)
ax2.set_ylabel('Accuracy', fontsize=14)
ax2.set_xticks(list(range(0, 50, 2)))  # Mostra i tick ogni 5 epochs per ridurre la sovrapposizione
ax2.set_ylim(0.20, 0.95)  # Fissa la scala dell'asse y tra 0.20 e 0.95
ax2.legend()
ax2.grid(True)


plt.tight_layout()
plt.savefig('/Users/jackvittori/Desktop/highnoise/training02.png', dpi=300)
plt.show()

In [46]:
early_probs_distribution = []
final_probs_distribution = []
for img, target in tqdm(validation_dataloader.dataset): 
    img = img / torch.linalg.norm(img).view(-1, 1)
    evaluation = fully_evaluation_ansatz(params=weights, state=img)
    early_full, final_full = evaluation
    
    early_prediction = torch.argmax(early_full, dim=1)
    early_probs = early_full[0, early_prediction].item()
    
    final_prediction = torch.argmax(final_full, dim=1)
    final_probs = final_full[0, final_prediction].item()
    
    early_probs_distribution.append(early_probs)
    final_probs_distribution.append(final_probs)

plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(10, 5))

# Creazione dell'istogramma per il primo array
ax.hist(early_probs_distribution, bins=60, density=False, alpha=0.5, color='g', edgecolor='black', label='Mid circuit probability distribution')

# Creazione dell'istogramma per il secondo array
ax.hist(final_probs_distribution, bins=60, density=False, alpha=0.5, color='r', edgecolor='black', label='final circuit probability distribution')

# Titolo e etichette degli assi
ax.set_title('Prediction confidence distribution', fontsize=16)
ax.set_xlabel('Prediction confidence', fontsize=14)
ax.set_ylabel('Occurrencies', fontsize=14)

# Aggiunta della legenda per distinguere le distribuzioni
ax.legend()

# Griglia e layout
ax.grid(True)
plt.tight_layout()

plt.savefig('/Users/jackvittori/Desktop/highnoise/probability_distribution02.png', dpi=300)
plt.show()

In [47]:
thresholds = [round(x * 0.01 + 0.26, 2) for x in range(26)]
summary, table = explain_evaluation(validation_dataloader, weights, thresholds)

In [48]:
ev_data = {
    'summary': summary,
    'table': table}

with open("/Users/jackvittori/Desktop/highnoise/evaluation02.pickle", "wb") as file:
    pickle.dump(ev_data, file)

# TRAINING LR = 001

# LOW

In [ ]:
import torch
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from data_utils import mnist_preparation, add_salt_and_pepper_noise 
from evaluationUtils import calculate_mcm_accuracy
from tqdm import tqdm
import matplotlib as plt
from mcmadaptablemodel import MCMQuantumModel, MCMCircuit
from pennylane import Device
from pennylane.measurements import StateMP
from torch.nn import Module, ParameterDict
import matplotlib.pyplot as plt
from OriginalModel import FullQuantumModel, QuantumCircuit
import warnings
from typing import Optional, Dict, List, Any
from torch.utils.data import DataLoader, dataloader
from time import time
import math
from pennylane.measurements import MidMeasureMP
torch.manual_seed(1234)

In [ ]:
model = MCMQuantumModel(qubits=8, layers=8, early_exits=[3])

In [ ]:
labels = [0,1,2,3]
# Download MNIST and prepare transforms
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                                transforms.Resize((16, 16)),  # Resize to 16x16
                                transforms.ToTensor(),
                                transforms.Lambda(lambda img: add_salt_and_pepper_noise(img, salt_prob=0.1, pepper_prob=0.1)),
                                transforms.Normalize((0.1307,), (0.3081,))  # Normalize
                             ]))
#train/vali/test 70/15/15 split, see data_utils.py for further details
train_dataloader, validation_dataloader, test_dataloader = mnist_preparation(dataset=mnist, labels = labels, train_test_ratio=0.7,batch_size=64, vali_test_ratio=0.5)

print("Images in the training set: ", len(train_dataloader.dataset), "\n Images in the validation set: ", len(validation_dataloader.dataset), "\n Images in the test set: ", len(test_dataloader.dataset))

In [ ]:
mcm_accuracy, fm_accuracy, loss_history = model.fit(dataloader=train_dataloader, learning_rate=0.01, epochs=40, show_plot=True)

In [ ]:
trained_params = model.params
data = {
    'model_params': trained_params,
    'mcm_accuracy': mcm_accuracy,
    'fm_accuracy': fm_accuracy,
    'loss_history': loss_history
}

with open("/Users/jackvittori/Desktop/lr001/lowtraining.pickle", "wb") as file:
    pickle.dump(data, file)

# MID TRAIN 001

In [ ]:
import torch
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from data_utils import mnist_preparation, add_salt_and_pepper_noise 
from evaluationUtils import calculate_mcm_accuracy
from tqdm import tqdm
import matplotlib as plt
from mcmadaptablemodel import MCMQuantumModel, MCMCircuit
from pennylane import Device
from pennylane.measurements import StateMP
from torch.nn import Module, ParameterDict
import matplotlib.pyplot as plt
from OriginalModel import FullQuantumModel, QuantumCircuit
import warnings
from typing import Optional, Dict, List, Any
from torch.utils.data import DataLoader, dataloader
from time import time
import math
from pennylane.measurements import MidMeasureMP
torch.manual_seed(1234)

In [ ]:
model = MCMQuantumModel(qubits=8, layers=8, early_exits=[3])

In [ ]:
labels = [0,1,2,3]
# Download MNIST and prepare transforms
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                                transforms.Resize((16, 16)),  # Resize to 16x16
                                transforms.ToTensor(),
                                transforms.Lambda(lambda img: add_salt_and_pepper_noise(img, salt_prob=0.15, pepper_prob=0.15)),
                                transforms.Normalize((0.1307,), (0.3081,))  # Normalize
                             ]))
#train/vali/test 70/15/15 split, see data_utils.py for further details
train_dataloader, validation_dataloader, test_dataloader = mnist_preparation(dataset=mnist, labels = labels, train_test_ratio=0.7,batch_size=64, vali_test_ratio=0.5)

print("Images in the training set: ", len(train_dataloader.dataset), "\n Images in the validation set: ", len(validation_dataloader.dataset), "\n Images in the test set: ", len(test_dataloader.dataset))

In [ ]:
mcm_accuracy, fm_accuracy, loss_history = model.fit(dataloader=train_dataloader, learning_rate=0.01, epochs=40, show_plot=True)

In [ ]:
trained_params = model.params
data = {
    'model_params': trained_params,
    'mcm_accuracy': mcm_accuracy,
    'fm_accuracy': fm_accuracy,
    'loss_history': loss_history
}

with open("/Users/jackvittori/Desktop/lr001/midtraining.pickle", "wb") as file:
    pickle.dump(data, file)

# HIGH LR 001

In [ ]:
import torch
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from data_utils import mnist_preparation, add_salt_and_pepper_noise 
from evaluationUtils import calculate_mcm_accuracy
from tqdm import tqdm
import matplotlib as plt
from mcmadaptablemodel import MCMQuantumModel, MCMCircuit
from pennylane import Device
from pennylane.measurements import StateMP
from torch.nn import Module, ParameterDict
import matplotlib.pyplot as plt
from OriginalModel import FullQuantumModel, QuantumCircuit
import warnings
from typing import Optional, Dict, List, Any
from torch.utils.data import DataLoader, dataloader
from time import time
import math
from pennylane.measurements import MidMeasureMP
torch.manual_seed(1234)

In [ ]:
model = MCMQuantumModel(qubits=8, layers=8, early_exits=[3])